# Introduction: Why Julia?
- Fast, comparable to C<br>
- User-friendly, similar to Python<br>
- A replacement for Matlab in numerical computation. And it's free<br>
- A front for solvers including CPLEX, Gurobi, Cbc, SCS, etc. 
# Drawbacks:
- Too many breaking changes<br>
- Some packages are not as powerful as their counterparts in other languages
# Suitable for:
- People who are looking for answers to the problems that cannot be solved by a single language, e.g. a less pricy Matlab, or a fast solver wrapper that is also easy to debug<br>
- Reducing the number of languages one uses for different tasks<br>
- Writing and sharing your own packages<br>
- Or just using it as a powerful and handy calculator! Like getting the determinant of a matrix: 
```
>>A=[1 2;3 4] 
>>det(A)
```
- AND those who take Integer Programming this semester, and want to better understand solver:)

# Basics
- Intalling a package, e.g. Gurobi: `>>Pkg.add("Gurobi")`
- Recommended IDE: Atom, Jupyter Notebook, JuliaBox (To open Jupyter Notebook, run the following in the interactive session: 
```
>>using IJulia 
>>notebook()
```
- Getting examples for a function directly in the interactive session, e.g. `? det()`

# Example 1: Quasi-Newton Method (BFGS)
Learning matrix operations in Julia through a quasi-Newton method example.<br>
We want to find out the stationary point of the following multi-variate function:
$$f(x)=(x_1-2)^4+(x_1-2x_2)^2$$


In [1]:
f(x1,x2)=(x1-2)^4+(x1-2x2)^2 #define functions (one of three ways)
df(x1,x2)=[4(x1-2)^3+2(x1-2x2);-4(x1-2x2)]
Hessian(x1,x2)=[12(x1-2)^2+2x1 -4;-4 8]

Hessian (generic function with 1 method)

In [2]:
inv(Hessian(2,3))#getting the inverse of a matrix. Don't need additional package

2×2 Array{Float64,2}:
 0.5   0.25
 0.25  0.25

In [5]:
#initializing variables
x=[0;0]
f_now=f(x[1],x[2]) #in julia, indices start from 1, instead of 0
f_prev=-Inf
df_now=df(x[1],x[2])
H=Hessian(x[1],x[2])
ϵ=1e-08 #cool feature: \epsilon then Tab to get the greek letter
α=1 
Nloop=100
k=0

while k<=Nloop #a while loop
    if abs(f_now-f_prev)>ϵ #using if
        k+=1
        f_prev=f_now
        df_prev=df_now
        d=-inv(H)*df_now
        x=x+α*d
        f_now=f(x[1],x[2])
        df_now=df(x[1],x[2])
        s=α*d
        y=df_now-df_prev
        H=H+y*y'/(y'*s)-(H*s*s'*H)/(s'*H*s) #use ' to get transpose of a matrix
    else 
        break
    end
end
if abs(f_now-f_prev)>ϵ
    println("BFGS fails to converge winthin 100 steps") #print text and change line
else
    @printf("BFGS converges within %d steps",k) #print formatted text. Doesn't change line automatically
    println("The optimal value is ", f_now)
end   

BFGS converges within 20 stepsThe optimal value is 1.7570295550248682e-9


In [8]:
#put everything inside function so we can change starting point easily
function f(m_x,m_loop,m_α)
        #initializing variables
    x=m_x
    f_now=f(x[1],x[2]) #in julia, indices start from 1, instead of 0
    f_prev=-Inf
    df_now=df(x[1],x[2])
    H=Hessian(x[1],x[2])
    ϵ=1e-08 #cool feature: \epsilon then Tab to get the greek letter
    α=m_α 
    Nloop=m_loop
    k=0

    while k<=Nloop #a while loop
        if abs(f_now-f_prev)>ϵ #using if
            k+=1
            f_prev=f_now
            df_prev=df_now
            d=-inv(H)*df_now
            x=x+α*d
            f_now=f(x[1],x[2])
            df_now=df(x[1],x[2])
            s=α*d
            y=df_now-df_prev
            H=H+y*y'/(y'*s)-(H*s*s'*H)/(s'*H*s) #use ' to get transpose of a matrix
        else 
            break
        end
    end
    if abs(f_now-f_prev)>ϵ
        println("BFGS fails to converge winthin 100 steps") #print text and change line
    else
        @printf("BFGS converges within %d steps",k) #print formatted text. Doesn't change line automatically
        println("The optimal value is ", f_now)
    end   
end

f (generic function with 3 methods)

In [9]:
f([1;1],100,1)

BFGS converges within 17 stepsThe optimal value is 3.815550033188953e-9


In [12]:
f([1;1],100,0.1) #fails to converge. Change m_loop to 1000

BFGS fails to converge winthin 100 steps


# Example 2: A Simple Optimization Problem
JuMP is very useful when solving simple optimization for homework/fun. Or when you want to check the feasibility of a linear system.
$$\begin{aligned}
\min\quad &x_1+x_2\\
s.t.\quad &x_1+3x_2\geq 1\\
 &x_1,x_2>=0
\end{aligned}
$$

In [2]:
#using packages
#JuMP is the package for optimization. Gurobi is the solver wrapper for the solver Gurobi
using JuMP, Gurobi 

In [15]:
#defining a model object, m
m=Model(solver=GurobiSolver())

#defining variables
@variables m begin
    x1>=0
    x2>=0
end

#defining objective
@objective(m,Min,x1+x2)

#defining constraints
@constraint(m,x1+3x2>=1)

solve(m)

println("Objective value: ", getobjectivevalue(m))
println("x2= ", getvalue(x2))

Academic license - for non-commercial use only
Optimize a model with 1 rows, 2 columns and 2 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.3333333e-01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  3.333333333e-01
Objective value: 0.3333333333333333
x2= 0.3333333333333333


# Example 3: Solving a Capacitated Facility Location Problem Using JuMP
(Based on an example in CPLEX installation folder, facility.cpp)<br>
Let $\mathcal{L}$ be the set of locations, $\mathcal{C}$ be the set of customers. $f_l$ is the fixed cost of opening a facility, $v_{cl}$ the variable cost of delivering from location l to customer c (per unit), $d_l$ is the capacity of a facility.
$$\begin{aligned}
\min\quad &\sum_{l\in\mathcal{L}}f_lx_l+\sum_{l\in\mathcal{L}}\sum_{c\in\mathcal{C}}v_{cl}y_{cl}\\
s.t.\quad &\sum_{l\in\mathcal{L}}y_{cl}=1, \quad\forall c\in\mathcal{C}\\
&\sum_{c\in\mathcal{C}}y_{cl}\leq d_l x_l, \quad\forall l\in\mathcal{L}\\
&x_l\in\{0,1\}, \quad \forall l\in\mathcal{L}\\
&0\leq y_{cl}\leq 1, \quad \forall l\in\mathcal{L},\forall c\in\mathcal{C}
\end{aligned}$$

In [6]:
using JuMP, Gurobi 
#data
d=[ 3, 1, 2, 4, 1]
fixed=[ 480, 200, 320, 340, 300]
v=[[ 24, 74, 31, 51, 84], 
 [ 57, 54, 86, 61, 68],
 [ 57, 67, 29, 91, 71],
 [ 54, 54, 65, 82, 94],
 [ 98, 81, 16, 61, 27],
 [ 13, 92, 34, 94, 87],
 [ 54, 72, 41, 12, 78],
 [ 54, 64, 65, 89, 89]]
C=1:length(v)
L=1:length(d)

m=Model(solver=GurobiSolver())
#variable block
@variables m begin
    x[l in L], Bin
    0<=y[c in C, l in L]<=1
end

@objective(m,Min,sum(fixed[l]*x[l] for l in L)+sum(v[c][l]*y[c,l] for c in C, l in L))

@constraints(m, begin
    first_c[c=C],sum(y[c,l] for l in L)==1
    [l=L], sum(y[c,l] for c in C)<=d[l]x[l]
end)

solve(m)

print(m) #printing out the formulation
println("Solving time: ",getsolvetime(m))
println("x: ",getvalue(x))

Academic license - for non-commercial use only
Optimize a model with 13 rows, 45 columns and 85 nonzeros
Variable types: 40 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 13 rows, 45 columns, 85 nonzeros
Variable types: 40 continuous, 5 integer (5 binary)

Root relaxation: objective 1.298000e+03, 12 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1298.00000    0    1          - 1298.00000      -     -    0s
H    0     0                    1430.0000000 1298.00000  9.23%     -    0s
     0     0 1313.00000    0    1 1430.00000 1313.00000  8.18%     -    0s
H    0     0                    1383.0000000 1313.00000  5.06%     -    0s
     0     0 1315.00000    0    1 1383.00000

# Quiz 1: Using Newton's Method for Unconstrained Optimization
Let Q=[3 1;2 4]. Please find the stationary point for the following unconstrained program:
$$
\begin{aligned}
f(x)=\min\quad \frac{1}{2}x^TQx
\end{aligned}
$$
Using step length $\alpha=1$, tolerance $\epsilon=1e-08$, does method converge? If yes, how many iterations does it take to converge?<br>
(Hint: $df(x)=[3x_1+x_2,x_1+4x_2]'$, $Hessian=Q$)

# Quiz 2: Getting the Marginal Cost of Constraints

# More Resources:
[1] Shuvomoy Das Gupta's GitHub repository contains materials from 2015&2016 (last four files): https://github.com/Shuvomoy/juliaopt-notebooks/tree/master/notebooks<br>
[2] Examples for optimization problems in the JuMP repository: https://github.com/JuliaOpt/JuMP.jl/tree/master/examples
